![http://feng.li/](logo.png)Brought to you by [Dr. Feng Li](http://feng.li)

**{Other Formats Available: [PDF](http://feng.li/python) | [HTML](http://feng.li/python) | [SLIDES](http://feng.li/python)}**

# What Is Web Scraping?

The automated gathering of data from the internet is nearly as old as the internet itself. Although web scraping is not a new term, in years past the practice has been more commonly known as screen scraping, data mining, web harvesting, or similar variations. General consensus today seems to favor web scraping, so that is the term I use throughout the book, although I also refer to programs that specifically traverse multiple pages as web crawlers or refer to the web scraping programs themselves as bots.

In theory, web scraping is the practice of gathering data through any means other than a program interacting with an API (or, obviously, through a human using a web browser). This is most commonly accomplished by writing an automated program that queries a web server, requests data (usually in the form of HTML and other files that compose web pages), and then parses that data to extract needed information.

In practice, web scraping encompasses a wide variety of programming techniques and technologies, such as data analysis, natural language parsing, and information security. Because the scope of the field is so broad, this book covers the fundamental basics of web scraping and crawling in Part I and delves into advanced topics in Part II. I suggest that all readers carefully study the first part and delve into the more specific in the second part as needed.

# Your First Web Scraper

## Let's try the toy first

In [2]:
from urllib.request import urlopen
html = urlopen('https://feng.li/python/')
print(html.read())

b'<!DOCTYPE html>\n<html lang="en-US" class="no-js">\n  <head>\n    <meta charset="UTF-8">\n    <meta name="viewport" content="width=device-width, initial-scale=1">\n    <link rel="profile" href="http://gmpg.org/xfn/11">\n        <script>(function(html){html.className = html.className.replace(/\\bno-js\\b/,\'js\')})(document.documentElement);</script>\n<title>Python &#8211; Dr. Feng Li</title>\n<link rel=\'dns-prefetch\' href=\'//feng.li\' />\n<link rel=\'dns-prefetch\' href=\'//fonts.googleapis.com\' />\n<link rel=\'dns-prefetch\' href=\'//s.w.org\' />\n<link rel="alternate" type="application/rss+xml" title="Dr. Feng Li &raquo; Feed" href="https://feng.li/feed/" />\n<link rel="alternate" type="application/rss+xml" title="Dr. Feng Li &raquo; Comments Feed" href="https://feng.li/comments/feed/" />\n\t\t<script type="text/javascript">\n\t\t\twindow._wpemojiSettings = {"baseUrl":"https:\\/\\/s.w.org\\/images\\/core\\/emoji\\/2.4\\/72x72\\/","ext":".png","svgUrl":"https:\\/\\/s.w.org\\/ima

The above doesn’t look so great. Below is better.

In [3]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://feng.li/python/')
bs = BeautifulSoup(html.read(), 'html.parser')
print(bs)

<!DOCTYPE html>

<html class="no-js" lang="en-US">
<head>
<meta charset="utf-8"/>
<meta content="width=device-width, initial-scale=1" name="viewport"/>
<link href="http://gmpg.org/xfn/11" rel="profile"/>
<script>(function(html){html.className = html.className.replace(/\bno-js\b/,'js')})(document.documentElement);</script>
<title>Python – Dr. Feng Li</title>
<link href="//feng.li" rel="dns-prefetch">
<link href="//fonts.googleapis.com" rel="dns-prefetch"/>
<link href="//s.w.org" rel="dns-prefetch"/>
<link href="https://feng.li/feed/" rel="alternate" title="Dr. Feng Li » Feed" type="application/rss+xml"/>
<link href="https://feng.li/comments/feed/" rel="alternate" title="Dr. Feng Li » Comments Feed" type="application/rss+xml"/>
<script type="text/javascript">
			window._wpemojiSettings = {"baseUrl":"https:\/\/s.w.org\/images\/core\/emoji\/2.4\/72x72\/","ext":".png","svgUrl":"https:\/\/s.w.org\/images\/core\/emoji\/2.4\/svg\/","svgExt":".svg","source":{"concatemoji":"https:\/\/feng.li\/wo

## The complete case

In [5]:
from urllib.request import urlopen
from bs4 import BeautifulSoup

html = urlopen('https://feng.li/python/')
bs = BeautifulSoup(html.read(), 'html.parser')
nameList = bs.findAll('div', {'class':'entry-content'})
for name in nameList:
    print(name.get_text())


This page servers as the primary guide for my students who want to learn Python.



## Web Scraping with `BeautifulSoup`

In [1]:
import logging
import requests
import sys
import urllib

from bs4 import BeautifulSoup
from collections import OrderedDict
from urllib.parse import urlencode

def get_list(comp, page):
    """Function to get  web list pages for a given company and page number.

    Args:
        comp: Company name.
        page: The page number.

    Returns:
        newsData: A dictionary with news title as its key and other details as values.

    """
    newsData = OrderedDict()
    href = 'http://search.sina.com.cn/?%s&range=title&c=news&num=20&col=1_7&page=%s' % (comp, page) # comp -> first %s; page -> 2nd %s; col=1_7 -> financial news in sina
    html = requests.get(href)
    # Parsing html
    soup = BeautifulSoup(html.content, 'html.parser')
    divs = soup.findAll('div', {"class": "r-info r-info2"})
    for div in divs:
        head = div.findAll('h2')[0]
        # News title
        titleinfo = head.find('a')
        title = titleinfo.get_text()
        # News url
        url = titleinfo['href']
        # Other info
        otherinfo = head.find('span', {"class": "fgray_time"}).get_text()
        source, date, time = otherinfo.split()
        # News abstract
        abstract = div.find('p', {"class": "content"}).get_text()
        newsData[title] = [date, source, abstract, url]
    return newsData



if __name__ == "__main__":
    compRawStr = '百度'
    # Dealing with character encoding
    comp = compRawStr.encode('gbk')
    d = {'q': comp}
    pname = urlencode(d)
    # Scraping and printing the first two pages
    for page in range(3)[1:]:
        newsData = get_list(pname, page)
        for ky in newsData:
            print('\001'.join([ky] + newsData[ky])) # "\001" as separator

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


美团与百度合作无人驾驶送外卖，首单或落地雄安。2018-04-19新浪财经美团与百度合作无人驾驶送外卖，首单或落地雄安。http://finance.sina.com.cn/7x24/2018-04-19/doc-ifzihneq1276594.shtml
百度、一汽等7家企业获准在重庆开展自动驾驶“路测”。2018-04-19新浪财经百度、一汽等7家企业获准在重庆开展自动驾驶“路测”。http://finance.sina.com.cn/7x24/2018-04-19/doc-ifzihneq1239009.shtml
美团与百度合作无人驾驶送外卖，首单或落地雄安2018-04-19新浪财经美团与百度合作无人驾驶送外卖，首单或落地雄安】百度内部工作人员透露，百度与美团已达成相关合作，将无人驾驶技术应用到送餐环节，并计划在雄安率先试点。百度与美团这一合作原 ...http://finance.sina.com.cn/7x24/2018-04-19/doc-ifzihneq0108513.shtml
百度Apollo发布2.5版：新增方案成本直降九成2018-04-19新浪财经百度Apollo发布2.5版：新增方案成本直降九成】4月19日，百度对外发布Apollo 2.5版本，围绕场景、成本、车型、性能等维度进行了多项升级。http://finance.sina.com.cn/7x24/2018-04-19/doc-ifzihnep9851070.shtml
基于百度Apollo，首汽约车上线CI车载智能系统2018-04-19新浪财经基于百度Apollo，首汽约车上线CI车载智能系统】4月19日，首汽约车CI车载智能硬件系统正式上线。目前已在大连、成都、厦门等城市上线测试，随后将在全国范围内推广。http://finance.sina.com.cn/7x24/2018-04-19/doc-ifzfkmth6997964.shtml
百度与一汽、奇瑞和北汽就网络安全实验室合作2018-04-19中国证券网中国证券网讯 据腾讯科技4月19日消息，百度与一汽、奇瑞和北汽就网络安全实验室合作。http://finance.sina.com.cn/roll/2018-04-19/doc-ifzihnep9300219.shtml

Some characters could not be decoded, and were replaced with REPLACEMENT CHARACTER.


百度朱光：人工智能将帮助人类更专注于创造与想象2018-04-11中国证券报-中证网彭扬）百度高级副总裁朱光11日在亚洲博鳌论坛2018年年会“让人工智能落地”分论坛上表示，如同十年前开始重视互联网+一样，未来越来越多的公司会用到AI，把每个公司变成一个“AI+公司 ...http://finance.sina.com.cn/roll/2018-04-11/doc-ifyuwqez8996417.shtml
力帆股份：与百度在无人驾驶上有深度合作2018-04-11中国证券网中国证券网讯 力帆股份11日在互动平台上回复投资者提问时表示，公司与百度、盼达在新能源车无人驾驶开发上有深度合作；与重庆快键新能源汽车有限公司在新能源汽车供销上有战略合作 ...http://finance.sina.com.cn/stock/t/2018-04-11/doc-ifyteqtq8064834.shtml
腾讯阿里百度京东等巨头竞相试水区块链2018-04-11上海证券报区块链技术在商业世界中的应用已悄然铺开。记者梳理发现，最近一段时间以来，腾讯、阿里、百度、京东、华为、顺丰、谷歌、Facebook、三星等巨头已陆续试水区块链技术应用。http://finance.sina.com.cn/blockchain/roll/2018-04-11/doc-ifyteqtq7776091.shtml
长城汽车与百度签署汽车智能互联领域战略合作2018-04-10新浪财经长城汽车与百度签署汽车智能互联领域战略合作】4月9日，长城汽车和百度签署战略合作备忘录，宣布双方将在车辆智能网联、自动驾驶、共享出行、大数据四大领域展开战略合作，打造车辆 ...http://finance.sina.com.cn/7x24/2018-04-10/doc-ifyuwqez8279061.shtml
长城与百度签署汽车智能互联领域战略合作2018-04-10界面长城与百度签署汽车智能互联领域战略合作 4月9日，长城汽车和百度签署战略合作备忘录，宣布双方将在车辆智能网联、自动驾驶、共享出行、大数据四大领域展开战略合作，打造车辆智 ...http://finance.sina.com.cn/stock/s/2018-04-10/doc-ifyuwqez8265337.shtm

In [3]:
import logging
import requests
import sys

from bs4 import BeautifulSoup




def get_body(href):
    """Function to retrieve news content given its url.

    Args:
        href: url of the news to be crawled.

    Returns:
        content: the crawled news content.

    """
    html = requests.get(href)
    soup = BeautifulSoup(html.content, 'html.parser')
    div = soup.find('div', {"id": "artibody"})
    paras = div.findAll('p')
    content = ''
    for p in paras:
        ptext = p.get_text().strip().replace("\n", "")
        content += ptext
    return content



if __name__ == "__main__":
    logging.getLogger().setLevel(logging.INFO)
    # Getting and printing content for each url in the crawled web list pages
    with open("data/baidu_list") as f:
        for line in f:
            title, date, source, abstract, href = line.strip().split('\001')
            # Printing progress onto console
            logging.info('Scraping ' + href)
            content = get_body(href)
            print('\001'.join([title, date, source, abstract, href, content]))

INFO:root:Scraping http://finance.sina.com.cn/roll/2017-07-14/doc-ifyiaewh9121960.shtml
INFO:root:Scraping http://finance.sina.com.cn/money/bank/bank_hydt/2017-07-13/doc-ifyiakwa4000736.shtml


打不过俄罗斯“百度” Uber向战斗民族认怂了2017-07-14一财网俄罗斯业务移交Yandex 据彭博社报道，Uber昨日宣布将其在俄罗斯等几个国家的业务以合并的形式移交给俄罗斯打车服务商Yandex.Taxi。http://finance.sina.com.cn/roll/2017-07-14/doc-ifyiaewh9121960.shtml继中国市场后，Uber又将撤出俄罗斯市场，而投资者猜测这家公司甚至有可能撤出东南亚及印度市场。俄罗斯业务移交Yandex据彭博社报道，Uber昨日宣布将其在俄罗斯等几个国家的业务以合并的形式移交给俄罗斯打车服务商Yandex.Taxi。Uber此次合并中卖掉的除了俄罗斯市场外还包括亚美尼亚、阿塞拜疆、白俄罗斯、格鲁吉亚和哈萨克斯坦的打车业务，同时获得Yandex的部分股份作为交换，这与之前Uber和滴滴打车的合并方式非常相似。双方将成立一家合资公司，Uber将向合资公司注资2.25亿美元，占股36.6%;Yandex将注资1亿美元，占股59.3%，而剩余4.1%则由员工持股。这笔交易会在今年的第四季度完成，这家尚未命名的新公司估值在37.3亿美元左右。Yandex是俄罗斯最大的搜索引擎公司，Yandex.Taxi是其于2011年在俄罗斯推出的打车服务，在俄罗斯的打车市场占据着主导地位。2014年，Uber进入俄罗斯市场，双方展开了一场激烈的争夺，如同此前在中国市场对战滴滴一样，Uber在俄罗斯也与Yandex.Taxi陷入了补贴大战中。Uber表示，在俄罗斯与Yandex.Taxi争夺市场令其过去三年亏损了约1.7亿美元。Yandex.Taxi在俄罗斯运营的时间比Uber登陆该国早三年，并展示出降低服务价格以吸引用户的强烈意愿。对于此次合作，Uber欧中非地区负责人Pierre表示，双方联手不仅有利于Uber和Yandex，对于整个地区的司机和乘客而言，都是好事。同时，合作可以帮助Uber继续打造可持续的全球业务。在中国的坎坷竞争路该交易是继退出中国市场后Uber又一次撤出其主要市场。在中国市场上，Uber为了滴滴出行抢占市场损失了大约20亿美元之后，决定将其市场份额交予滴滴打车并得到滴滴出行17.5%的股份。Uber在2014年2月13日宣布正式进入中国市场。2013年8月，Uber正式在上海试运营。2014

INFO:root:Scraping http://finance.sina.com.cn/china/dfjj/2017-07-12/doc-ifyiamif2703041.shtml


百度金融弯道超车2017-07-13陆家嘴x3000;　百度金融弯道超车　　文/本刊记者 何映霏 摄影/OVE　　“人工智能和大数据给互联网金融的冲击，给像百度这样的有技术优势的公司弯道超车的机会。http://finance.sina.com.cn/money/bank/bank_hydt/2017-07-13/doc-ifyiakwa4000736.shtml基金经理老鼠仓，说好保本变巨亏，买基金被坑请到【基金曝光台】！信用卡无故遭盗刷，银行存款变保险，理财被骗请猛戳【金融曝光台】！百度金融弯道超车文/本刊记者 何映霏  摄影/OVE“人工智能和大数据给互联网金融的冲击，给像百度这样的有技术优势的公司弯道超车的机会。”百度副总裁、负责消费金融业务的黄爽在接受《陆家嘴》杂志采访时表示。技术基因全面注入百度金融BAT及京东之所以被称为互联网巨头，是由于垄断了70%以上的用户时间。巨头们一手掌握着流量，一手掌握着流量背后所代表的场景，而流量和场景又恰好是金融变现最好的方式之一，这也成为了互联网巨头向金融领域高歌猛进的充足理由。以支付宝花呗与京东白条为代表的电商场景消费金融，以电商平台为依托，支付作为入口，最终在电商平台上产生消费，消费过程中又接入各种金融产品，如消费分期等，这在无形中培养了广大受众的用户习惯。目前除了支付宝有蚂蚁花呗，京东有白条，百度研发了有钱花，苏宁有任性付，快钱有快易花。在互联网金融1.0时代，一切以场景为王。无论是线上还是线下，信用分期服务都依托于商业机构或者电商、服务交易的特定场景。但是到了互联网金融2.0时代，由于移动时代智能手机的发展，网络背后的一个个主体能够被识别。再加上用户在网络上沉淀的各种数据，其个体画像可以被精准刻画。近几年，互联网金融的竞争已经从资金端向资产端倾斜，黄爽看来，谈及资产端线上化时，需解决三个问题：一是如何通过线上触达需求方；二是核实需求方的真实身份；三是你需要给他匹配什么样的产品形态。“百度擅长于做第一和第三点，通过搜索引擎了解他人诉求，搜索到的信息点才是最真实的数据，朋友圈里发的可能只是你想成为的自己，甚至有故意经营的印记。” 黄爽谈道，百度以搜索立身，不仅拥有风控模型，还深挖了复杂的需求模型和响应模型，例如人脸识别系统就可以帮助百度消费金融高效地核实身份、反欺诈，还能降低人工审核时间，缩短审核周期，使消

INFO:root:Scraping http://finance.sina.com.cn/roll/2017-07-12/doc-ifyhwehx5784774.shtml


...

百度首次公布AI生态战略：无人驾驶开发时间仅用三天2017-07-06第一财经日报李彦宏乘无人车上五环 百度自动驾驶的两年和三天 刘佳 7月5日上午11点，北京五环路上车流拥挤，一辆行驶中的红色商务车看上去并没有什么特别。http://finance.sina.com.cn/roll/2017-07-06/doc-ifyhwefp0106577.shtml李彦宏乘无人车上五环 百度自动驾驶的两年和三天刘佳7月5日上午11点，北京五环路上车流拥挤，一辆行驶中的红色商务车看上去并没有什么特别。当时，百度创始人李彦宏正坐在副驾驶上，主驾驶位的司机双手未触碰方向盘，双脚离开油门刹车区域，车辆正在自动驾驶中。“现在车非常多，但是还是很平稳，感觉非常不错。前面也有一个屏幕可以展示出来自动驾驶汽车探索的周边的情况，我们的车处在自动驾驶的状态。”视频连线中，李彦宏和几公里外百度AI开发者大会现场的百度集团总裁陆奇分享了实时体验。当外界还在因为互联网格局变化议论纷纷时，百度正在用一次“把无人车开上五环”、“把智能驾驶能力开放出来”的行动，撕下过去“搜索公司”的标签，告诉外界：百度已经不再是一家互联网公司，它正在AllinAI。Allin人工智能这是百度第一次对外公布完整的AI生态开放战略。“AI将和前三次工业革命一样，让人类进入全新的时代。”陆奇这样说。在他看来，只做硬件或者软件都是不够的，AI技术与其他技术的不同之处，在于数据起到了重要而特殊的作用，必须系统加数据、通过学习、训练才能获得能力。“这次人工智能革命的本质是一场知识的革命，是快速、自动地来获取知识。”百度公布的AI开放生态战略包括：前端的对话式人工智能系统DuerOS和自动驾驶Apollo开放平台，以及后端的百度大脑和智能云。其中，DuerOS与Apollo开放平台都运用了百度大脑的核心能力。陆奇指出，百度已构建包含算法层、感知层、认知层和平台层技术架构的AI技术平台，并宣布百度大脑将全面开放60项核心AI能力，其中包括语音、视频、增强现实、机器人视觉、自然语音处理五大类14项新能力。在陆奇看来，中国科技公司在AI时代有着很大的优势。因为现在全世界有43%人工智能论文是华人撰写的，中国有足够大的技术累积，此外，中国的互联网公司在AI核心技术有了早期的着陆和长期的研发，有大量的资金，有世界上最大、最活跃的用户群，

INFO:root:Scraping http://finance.sina.com.cn/chanjing/gsnews/2017-07-06/doc-ifyhvyie0284926.shtml


百度打造开放式AI生态 开发者对如何盈利存疑虑2017-07-0621世纪经济报道百度打造开放式AI生态 开发者追捧与疑虑并存 本报记者 杨清清 北京报道 导读 科技巨头之所以开源技术、免费共享，是因为当前技术虽然领先但不够成熟，需要更多的人来参与推动。http://finance.sina.com.cn/roll/2017-07-06/doc-ifyhweua4040601.shtml百度打造开放式AI生态 开发者追捧与疑虑并存本报记者 杨清清 北京报道导读科技巨头之所以开源技术、免费共享，是因为当前技术虽然领先但不够成熟，需要更多的人来参与推动。7月5日早上9点，距离2017百度开发者大会正式开始还有1个小时的时间，大会现场展台区已人头攒动。大会正式开始后，不仅会场坐席爆满，在会场偏后的看台区域，也有很多找不到座位、直接坐在台阶上的参会者。推动开发者们热情高涨的，是百度的人工智能开放平台计划。在过去，包括谷歌、微软、苹果、Facebook等科技巨头每年均召开一次开发者大会，将自身的技术实力，尤其是近年来爆红的人工智能技术全面赋能全球开发者。在国内，2017百度开发者大会尚属首例。“AI的未来属于开发者，百度的将来必须建立在每个开发者共赢的基础上。”在大会现场，百度集团总裁兼COO陆奇表示，“AI时代机会极多，但开发壁垒同样很高，百度希望为每一位AI开发者解除障碍，提供工具、数据、基础设施以及良好的开发环境。”打造AI生态对于开发者而言，如何将原本需要数月时间的新车型移植及整车测试，压缩在三天内完成？如何用1分钟的时间，让原本只会英文语音交互的音箱，马上变得能够听懂中国话？答案很简单：借势。而百度正在试图与其他科技巨头一样，将自己所具备的人工智能之“势”，赋予相关的开发者。“百度AI生态战略的方向，是建立和引领新一代的AI计算平台，通过将DuerOS、阿波罗平台、百度大脑和百度智能云等综合在一起，形成一个强大的AI生态。”陆奇指出。其中，对话式人工智能系统和自动驾驶两大领域的开放计划，成为本次百度开发者大会的关注重心。所谓对话式人工智能系统，即DuerOS系统，百度将此系统能力进行开放后，开发者能够在基础技术框架上构建匹配自身需求的语音交互系统，并搭载到相应的智能硬件上。据百度度秘事业部总经理景鲲介绍，DuerOS开放平台运用了最先进的自然语言处理技术，

INFO:root:Scraping http://finance.sina.com.cn/roll/2017-07-06/doc-ifyhvyie0279853.shtml


百度无人驾驶汽车直播中压实线变道 该罚李彦宏吗？2017-07-06大洋网-广州日报百度视频展示无人驾驶汽车 不打转向灯 压实线变道 引交管部门介入调查 无人驾驶违章 该罚李彦宏吗？大洋网-广州日报 邓莉 昨日的2017百度AI开发者大会上，百度董事长兼首席执行官李彦宏 ...http://finance.sina.com.cn/chanjing/gsnews/2017-07-06/doc-ifyhvyie0284926.shtml百度视频展示无人驾驶汽车 不打转向灯 压实线变道 引交管部门介入调查无人驾驶违章 该罚李彦宏吗？大洋网-广州日报 邓莉昨日的2017百度AI开发者大会上，百度董事长兼首席执行官李彦宏通过视频直播展示了一段自己乘坐公司研发的无人驾驶汽车的情景。视频中，李彦宏坐在一辆红色汽车的副驾驶位上，他称自己刚刚上五环，正在前往会场的路上，“车处在自动驾驶的状态”，整段视频约1分钟长。 @视觉中国文/广州日报全媒体记者邓莉以互联网搜索引擎起家的百度，当前一个重要的发展方向是AI与无人驾驶汽车。昨日，在百度AI开发者大会上，百度创始人、董事长兼首席执行官李彦宏通过视频直播展示了一段自己乘坐百度研发的无人驾驶汽车的情景。不过，这本应是一次尖端技术演示，却引发了一场争议——细心的网友通过视频发现，这辆无人驾驶的车辆在“众目睽睽”之下违规了，随后北京交警表示将介入调查，那么问题来了：在司机没有驾驶的情况下违章，该罚谁？无人驾驶汽车直播中压实线变道广州日报全媒体记者查看该场发布会的回顾视频时留意到，在让现场无数人为之鼓掌的视频连线中，李彦宏坐在副驾驶位，驾驶员全程不碰方向盘，汽车自动行驶，顺利开上了北京的北五环，此时车辆在画有实线的路段并到另一车道，同时也没有打转向灯。这段视频画面被网友转发后迅速发酵，随后北京交管局也表示已经获悉相关情况，正在就此展开调查。在交管部门未公布调查结果之前，昨天下午，百度对外补充解释道，李彦宏所乘坐的无人驾驶汽车，驾驶位有人乘坐，为百度智能汽车事业部总经理顾维灏，而顾维灏全程也未用双手触碰方向盘。关于百度这辆无人驾驶汽车的违章问题，业内争议焦点在于：无人驾驶汽车该不该配备驾驶员？驾驶员需要什么资质？百度这辆演示车的违章情况，该处罚谁？无人驾驶处罚成难题？默认规则看“车辆控制权”如果是一辆由驾驶员正常驾驶的汽车实线变道，依据《

# Web Crawling with `Scrapy`

One of the challenges of writing web crawlers is that you’re often performing the same tasks again and again: find all links on a page, evaluate the difference between internal and external links, go to new pages. These basic patterns are useful to know and to be able to write from scratch, but the Scrapy library handles many of these details for you.

##  Installing Scrapy

- After Anaconda is installed, you can install Scrapy by using this command:
   
      conda install -c conda-forge scrapy



## Dealing with Different Website Layouts

Fortunately, in most cases of web crawling, you’re not looking to collect data from sites you’ve never seen before, but from a few, or a few dozen, websites that are pre-selected by a human. This means that you don’t need to use complicated algorithms or machine learning to detect which text on the page “looks most like a title” or which is probably the “main content.” You can determine what these elements are manually.

The most obvious approach is to write a separate web crawler or page parser for each website. Each might take in a URL, string, or BeautifulSoup object, and return a Python object for the thing that was scraped.


## Initializing a New Spider

To create a new spider in the current directory, run the following from the **command line (NOT THE PYTHON PROMPT)**:
```
    scrapy startproject wikiSpider
```    
    
This creates a new subdirectory in the directory the project was created in, with the title wikiSpider. Inside this directory is the following file structure:

- scrapy.cfg
- wikiSpider
  - spiders
     - __init.py__
  - items.py
  - middlewares.py
  - pipelines.py
  - settings.py
  - __init.py__
  
  
## Generate some spiders with templates from the command line

    scrapy genspider example example.com 
    scrapy genspider example2 example.com 
    scrapy genspider example3 example2.com 
    
  
## Writing a Simple Scraper

To create a crawler, you will add a new file inside the spiders directory at wikiSpider/wikiSpider/spiders/article.py. In your newly created **article.py** file, write the following:

```python
    import scrapy

    class ArticleSpider(scrapy.Spider):
        name='article'

        def start_requests(self):
            urls = [
                'http://en.wikipedia.org/wiki/Python_%28programming_language%29',
                'https://en.wikipedia.org/wiki/Functional_programming',
                'https://en.wikipedia.org/wiki/Monty_Python']
            return [scrapy.Request(url=url, callback=self.parse) for url in urls]

        def parse(self, response):
            url = response.url
            title = response.css('h1::text').extract_first()
            print('URL is: {}'.format(url))
            print('Title is: {}'.format(title))
```

## Run this article spider

You can run this article spider by navigating to the wikiSpider/wikiSpider directory and running from the command line:

    scrapy runspider article.py
        
## Run your project with at the project root directory

    scrapy crawl table -o table.csv  --logfile table.log
    

## Scrapy Shell

To do the crawler interactively, just run from the command line

    scrapy shell "http://en.wikipedia.org/wiki/Python_%28programming_language%29"

# Lab 

Use `scrapy` framework to implement the we studied with `BeautifulSoup`